In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np 
import cvxpy as cvx 
# from gurobipy import * 
from off_pol_eval_functions import * 


from scipy.optimize import minimize
import datetime as datetime
import pickle
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import collections as matcoll
from sklearn import svm
import sys
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

np.random.seed(2)
def multipage(filename, figs=None, dpi=200):
    pp = PdfPages(filename)
    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]
    for fig in figs:
        fig.savefig(pp, format='pdf')
    pp.close()

d = 10  # dimension of x 
n = 2000; 
mu_x = np.zeros(d); 
sigma_x = np.random.normal(size = (d,1))
sigma_x += np.abs(np.min(sigma_x))+0.5
sigma_x = np.multiply(sigma_x, np.eye(d))
sigma_x /= 2# normalize covariances a little bit 

W = 1.5 #treatment effect
# interact_x = 2
white_noise_coef = 0.1


# generate propensity model 
def real_prop(x, beta_prop): 
    T_SIG = 5
    if len(x.shape) > 1: 
        n= x.shape[1]
    else:
        n= len(x)
    return np.dot(x, beta_prop) + np.random.normal(size = (n,1))*T_SIG
    # T is normally distributed conditional on covariates 
    
# coefficient of treatment effect
beta_cons = -5
beta_x = np.random.normal(size = (d,1))
# interaction term with treatment 
beta_x_T = np.random.normal(size = (d,1))*1.5

# sparse interaction terms 
sparse_entries = np.random.choice(range(d),size =  int(round(0.7*d)),replace = False)
beta_x_T[sparse_entries] = 0     
sparse_entries = np.random.choice(range(d),size =  int(round(0.35*d)),replace = False)
beta_x[sparse_entries] = 0  

FREQ = 20 
beta_x_quad_T = np.random.normal(size = (d,1))
sparse_entries = np.random.choice(range(d),size =  int(round(0.65*d)),replace = False)
beta_x_quad_T[sparse_entries] = 0

TRUE_PROP_BETA = np.asarray(beta_x_quad_T + np.random.normal( loc= np.ones((d,1))*2, size = (d,1))).flatten()
print TRUE_PROP_BETA
def real_risk(T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x):    
    n = len(T); risk = np.zeros(n)
    if np.isscalar(T):
        risk = T*beta_cons + np.dot(beta_x.T, x) + np.dot(beta_x_T.T, x*T) + (T-np.dot(beta_x_quad_T.T,x))**2
    else: 
        for i in range(len(T)): 
            risk[i] = T[i]*beta_cons + np.dot(beta_x.T, x[i,:]) + np.dot(beta_x_T.T, x[i,:]*T[i]) + (T[i]-np.dot(beta_x_quad_T.T,x[i,:]))**2#+ np.dot(beta_x_quad_T.T, (x[i,:]**2)*T[i]) + np.dot(beta_x_high_freq.T, np.sin(x[i,0:HIGH_FREQ_N]*FREQ)*T[i])
    return risk

T_SIG = 4
def generate_data(mu_x, sigma_x_mat, n, beta_cons, beta_x, beta_x_T): 
#     x = np.random.normal(mu_x, sigma_x, size = n)
    # generate n datapoints from the same multivariate normal distribution
    x = np.random.multivariate_normal(mean = mu_x, cov= sigma_x_mat, size = n ) 
    print x.shape 
    print "xshape"
    T = np.random.normal(0, T_SIG, n) + np.dot(x, TRUE_PROP_BETA) + 2*x[:,1] + 4*x[:,2] - 2*x[:,3]
    true_resid = T - np.dot(x, TRUE_PROP_BETA)
    true_Q = norm.pdf( T - np.dot(x, TRUE_PROP_BETA), loc = 0, scale = T_SIG )
    y_sigma = 0.5
    white_noise_coef = 5
    
    clf = LinearRegression(); clf.fit(x, T)
    y_hat = clf.predict(x)
    Y = np.zeros(n)
    for i in range(n): 
        Y[i] = T[i]*beta_cons + np.dot(beta_x.T, x[i,:]) + T[i]*np.dot(beta_x_T.T, x[i,:]) + (T[i] - np.dot(beta_x_quad_T.T,x[i,:]))**2 #+ np.dot(beta_x_quad_T.T, (x[i,:]**2)*T[i]) + np.dot(beta_x_high_freq.T, np.sin(x[i,0:HIGH_FREQ_N]*FREQ)*T[i])
    Y += np.random.multivariate_normal(mean = np.zeros(n), cov=white_noise_coef * np.eye(n))
    # get pdf from residuals 
    resid = Y - y_hat
    # get norm pdf 
    Q = norm.pdf(resid, loc = np.mean(resid), scale=np.std(resid))
    T = T.flatten()
    return [x, T, Y, true_Q, clf]

[x_full, T_full, Y_full, true_Q_full, clf] = generate_data(mu_x, sigma_x, n, beta_cons, beta_x, beta_x_T)

#compute real risk 
print np.mean( real_risk(T_full, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_full))

plt.hist(Y_full); plt.title('Y')
plt.figure()
plt.title('T')
plt.hist(T_full)
Q = true_Q_full
plt.figure()
plt.hist(true_Q_full)


ImportError: No module named cvxpy

## In this setting the optimal treatment is similar to the observed treatment policy. 

In [ ]:
cutoff = 200
x_full=x_full[Y_full < cutoff]
T_full=T_full[Y_full < cutoff]
true_Q_full=true_Q_full[Y_full < cutoff]
Y_full = Y_full[Y_full < cutoff]

plt.hist(Y_full); plt.title('Y')
plt.figure()
plt.title('T')
plt.hist(T_full)
Q = true_Q_full
plt.figure()
plt.hist(true_Q_full)

## Reserve half for out of sample evaluation



In [ ]:
n = 1000
x_test = x_full[n:]; Y_test = Y_full[n:]; T_test = T_full[n:]; true_Q = true_Q_full[n:]
x = x_full[0:n]; Y = Y_full[0:n];T = T_full[0:n]; true_Q = true_Q_full[0:n]


## Optimal treatment tends to bounds

In [ ]:

# import pickle

# pickle.dump(x_full, open(str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")) + 'x.p', 'wb'))
# pickle.dump(T_full, open(str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")) + 'T.p', 'wb'))
# pickle.dump(Y_full, open(str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")) + 'Y.p', 'wb'))
# pickle.dump(true_Q_full, open(str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")) + 'Q.p', 'wb'))


In [ ]:
cons_T = np.linspace(-15,15,20)

plt.scatter(cons_T,  [np.mean( real_risk(np.ones(n)*t, beta_cons, beta_x, beta_x_T, beta_x_quad_T,x)) for t in cons_T ] )

def real_risk(T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x):    
    if np.isscalar(T):
        risk = T*beta_cons + np.dot(beta_x.T, x) + np.dot(beta_x_T.T, x*T) + (T-np.dot(beta_x_quad_T.T,x))**2
    else: 
        risk = np.zeros(len(T))
        for i in range(len(T)): 
            risk[i] = T[i]*beta_cons + np.dot(beta_x.T, x[i,:]) + np.dot(beta_x_T.T, x[i,:]*T[i])+ (T[i]-np.dot(beta_x_quad_T.T,x[i,:]))**2 #+ np.dot(beta_x_quad_T.T, (x[i,:]**2)*T[i]) + np.dot(beta_x_high_freq.T, np.sin(x[i,0:HIGH_FREQ_N]*FREQ)*T[i])
    return risk
def real_risk_scalar(T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x):    
    return T*beta_cons + np.dot(beta_x.T, x) + np.dot(beta_x_T.T, x*T) + (T-np.dot(beta_x_quad_T.T,x))**2

bounds = [(min(T)/(0.5*d) , max(T)/np.abs(0.5*d)  ) for i in range(d) ]
bnds = tuple(tuple(x) for x in bounds)

def get_oracle_treatment(beta_cons, beta_x, beta_x_T, beta_x_quad_T, x): 
    res = minimize(real_risk_scalar, np.mean(T), args =(beta_cons, beta_x, beta_x_T, beta_x_quad_T,x))
    return res.x 

oracle_T = [ get_oracle_treatment(beta_cons, beta_x, beta_x_T, beta_x_quad_T, x[i,:]) for i in range(n)]
plt.hist(np.asarray(oracle_T))

fig=plt.figure()
for i in range(50): 
    plt.plot(cons_T, np.asarray([ real_risk(t, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x[i,:]) for t in cons_T ]), alpha = 0.3 )

for i in range(50): 
    plt.scatter(T[i], real_risk(T[i], beta_cons, beta_x, beta_x_T, beta_x_quad_T, x[i,:]),color='r')
plt.xlabel('T')
plt.ylabel('y (loss)')
# plt.axis('off')


fig.savefig('outcome-model.pdf')

# Constant policy 

In [ ]:
# real_risk(disc_res,  )
# disc_res

In [ ]:
LAMBDA = -3

def pol_eval_wrapper(beta, args): 
    data = dict(args)
    t_lo = data['t_lo']; t_hi = data['t_hi']
    tau = np.clip(np.dot(beta, data['x'].T) , t_lo, t_hi).flatten()
    data['tau'] = tau
    return data['sgn']*(off_policy_evaluation(**data))

def pol_eval_l2_wrapper(beta, args): 
    data = dict(args)
    t_lo = data['t_lo']; t_hi = data['t_hi']
    tau = np.clip(np.dot(beta, data['x'].T) , t_lo, t_hi).flatten()
    data['tau'] = tau
    return data['sgn']*(off_policy_evaluation(**data) + LAMBDA * np.linalg.norm(beta,2))


def pol_eval_l1_wrapper(beta, args): 
    data = dict(args)
    t_lo = data['t_lo']; t_hi = data['t_hi']
    tau = np.clip(np.dot(beta, data['x'].T) , t_lo, t_hi).flatten()
    data['tau'] = tau
    return data['sgn']*(off_policy_evaluation(**data) + LAMBDA * sum(abs(beta)))

def off_policy_evaluation(**params):
    """
    Takes in a choice of kernel and dictionary of parameters and data required for evaluation
    tau is a vector of treatment values (assumed given)
    If y_samp, T_samp is present, use that instead. 
    """
    THRESH = params['threshold']
    y_out = params['y']; x = params['x']; h = params['h'];Q = params['Q']; n = params['n']; t_lo = params['t_lo'];  t_hi = params['t_hi']
    kernel = params['kernel_func'];kernel_int =  params['kernel_int_func']
    if ('y_samp' in params.keys()):
        y_out = params['y_samp']
    if ('T_samp' in params.keys()): 
        T = params['T_samp']
    else: 
        T = params['T']
    if ('x_samp' in params.keys()):
        x = params['x_samp']
    loss = 0
    tau = params['tau']
    clip_tau = np.clip(tau, t_lo, t_hi)
    for i in np.arange(n): 
        Q_i = Q(x[i,:], T[i], t_lo, t_hi)
        if (abs(clip_tau[i] - t_lo) <= h):
            alpha = kernel_int((t_lo-clip_tau[i])/h, 1)
            if alpha < 0.5: 
                print alpha
        elif (abs(clip_tau[i] - t_hi) <= h):
            alpha = kernel_int(-1,  (t_hi - clip_tau[i])/h )
            if alpha < 0.5: 
                print alpha
        else:
            alpha = 1
        loss += kernel( (clip_tau[i] - T[i])/h )*1.0 * y_out[i]/max(Q_i,THRESH) * 1.0/alpha
    return loss/(1.0*h*n)


def off_pol_epan_lin_l2_grad(beta, *args):
    """
    Compute a gradient for special case of Epanechnikov kernel and linear policy tau
    """
    # THRESH = 0.001
    d = len(beta) 
    params = dict(args[0])
    #! FIXME x vs xsamp
    tau = np.dot(beta, params['x'].T)
    params['tau'] = tau
    params['beta'] = beta

    THRESH = params['threshold']

    [f, g, nabla_f, nabla_g] = f_g(**params)
    # compute gradient vector via quotient rule
    if g < THRESH: 
        g = THRESH  
    
    return params['sgn']*(np.asarray((g*nabla_f - f*nabla_g) / g**2 ) + LAMBDA * 2*beta)

def off_pol_epan_lin_l1_grad(beta, *args):
    """
    Compute a gradient for special case of Epanechnikov kernel and linear policy tau
    """
    d = len(beta) 
    params = dict(args[0])
    #! FIXME x vs xsamp
    tau = np.dot(beta, params['x'].T)
    params['tau'] = tau; params['beta'] = beta
    THRESH = params['threshold']
    [f, g, nabla_f, nabla_g] = f_g(**params)
    # compute gradient vector via quotient rule
    if g < THRESH: 
        g = THRESH  
    l1_sub = np.zeros(d)
    for i in range(d): 
        if beta[i] < 0: 
            l1_sub[i] = -1 
        elif beta[i] > 0: 
            l1_sub[i] = 1
        else: 
            l1_sub[i] = 0 
    return params['sgn']*(np.asarray((g*nabla_f - f*nabla_g) / g**2 ) + LAMBDA * l1_sub)

def f_g(**params): 
    THRESH = params['threshold']
    y_out = params['y']; x = params['x']; h = params['h'];Q = params['Q']; n = params['n']; t_lo = params['t_lo'];  t_hi = params['t_hi']
    kernel = params['kernel_func'];kernel_int =  params['kernel_int_func']
    if ('y_samp' in params.keys()):
        y_out = params['y_samp']
    if ('T_samp' in params.keys()): 
        T = params['T_samp']
    else: 
        T = params['T']
    if ('x_samp' in params.keys()):
        x = params['x_samp']
    BMI_IND = params.get('BMI_IND') # propensity score for warfarin data evaluations 
        
    loss = 0
    g = 0 # also keep track of normalized probability ratio quantity 
    partial_f = 0 
    partial_g = 0 
    tau = params['tau']
    clip_tau = np.clip(tau, t_lo, t_hi)
    Qs = np.zeros(n)
    for i in np.arange(n): 
        if (params.get('DATA_TYPE') == 'warfarin'): 
            Q_i = Q(x[i,BMI_IND], T[i], t_lo, t_hi)
        else: 
            Q_i = Q(x[i], T[i], t_lo, t_hi)
        if (abs(clip_tau[i] - t_lo) <= h):
            alpha = kernel_int((t_lo-clip_tau[i])/h, 1)
        elif (abs(clip_tau[i] - t_hi) <= h):
            alpha = kernel_int(-1,  (t_hi - clip_tau[i])/h )
        else:
            alpha = 1
        Qs[i] = kernel( (clip_tau[i] - T[i])/h )/max(Q_i,THRESH)
        loss += kernel( (clip_tau[i] - T[i])/h )*1.0 * y_out[i]/max(Q_i,THRESH) * 1.0/alpha
        if abs((clip_tau[i] - T[i])/h) >= 1:
            partial_f += 0 # don't add anything to partial derivatives 
        else:
            partial_g += -1.5 * ((clip_tau[i] - T[i])/h ) * 1.0/max(Q_i,THRESH) * x[i,:]
            partial_f += -1.5 * ((clip_tau[i] - T[i])/h ) * y_out[i]/max(Q_i,THRESH) * x[i,:]
    norm_sum = np.mean(Qs)
    return [loss/(1.0*h*n), 1.0*norm_sum/h, partial_f/(1.0*n*h**2) , partial_g/(1.0*n*h**2) ]




In [ ]:
def oracle_eval_wrapper(beta, args): 
    data = dict(args)
    t_lo = data['t_lo']; t_hi = data['t_hi']
    tau = np.clip(np.dot(beta, data['x'].T) , t_lo, t_hi).flatten()
    x = data['x']
    return np.mean(real_risk(tau, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x))



In [ ]:
## Train GPS on your simulated data
def norm_T_Q(x,t,t_lo,t_hi): 
    return norm.pdf( t - np.dot(x, TRUE_PROP_BETA), loc = 0, scale = T_SIG )

trainind = np.random.choice(range(n),size = int(round(0.4*n)),replace = False)
train = x[trainind]
test_mask = np.ones(n, dtype=bool)
test_mask[trainind] = False
test = x[test_mask,:]
t_lo = min(T)
t_hi = max(T) 
d = x.shape[1]
train_data = { 'n': train.shape[0],'h': 4, 'y': Y[trainind],'Q': norm_T_Q,'x_full': x[trainind], 'x': x[trainind],'x_samp': x[trainind], 'T_samp': T[trainind], 'd': d, 'T': T[trainind],'t_lo': t_lo ,'t_hi': t_hi  }
train_data['kernel_int_func'] = epanechnikov_int
train_data['T_samp'] = np.array(T[trainind],order='F'); train_data['y_samp'] = Y[trainind]
train_data['kernel_func'] = epanechnikov_kernel
train_data['inds'] = trainind
train_data['threshold'] = 0.01


In [ ]:

def discretize_tau_policy(**params):
    '''
    Discretize the treatment vector 'tau' according to uniform binning.
    '''
    x = params['x_samp']
    T = params['T_samp']
    n_bins = params['n_bins']
    t_lo = min(T)
    t_hi = max(T)
    bins = np.linspace(t_lo, t_hi, n_bins)
    T_binned = np.digitize(T, bins).flatten()
    bin_means = [T[T_binned == i].mean() for i in range(1, n_bins)]
    tau_binned = np.digitize(params['tau'], bins).flatten()
    return tau_binned

def off_pol_disc_evaluation(policy, **params):
    THRESH = params['threshold']
    y_out = params['y']; x = params['x_samp']; h = params['h']; Q = params['Q']; n = params['n']; t_lo = params['t_lo']; t_hi = params['t_hi']
    n_bins = params['n_bins']
    if ('y_samp' in params.keys()):
        y_out = params['y_samp'].flatten()
    if ('T_samp' in params.keys()):
        T = params['T_samp'].flatten()
    else:
        T = params['T'].flatten()

    t_lo = min(T)
    t_hi = max(T)
    bin_width = t_hi-t_lo
    bins = np.linspace(t_lo, t_hi, n_bins)
    T_binned = np.digitize(T, bins, right = True).flatten()
    bin_means = [T[T_binned == i].mean() for i in range(1, len(bins))]

    loss = 0
    tau_vec = policy(**params).flatten()

    i=0
    #! FIXME need to establish whether policy returns discrete bins or means
    treatment_overlap = np.where(np.equal(tau_vec.flatten(), T_binned))[0]
    n_overlap = len(treatment_overlap)
    Qs = np.zeros(n_overlap)
    for ind in treatment_overlap:
        Q_i = Q(x[ind], bin_means[T_binned[ind]-1], t_lo, t_hi) * bin_width*1.0/n_bins # BUG FIX: this is going to have to be integrated against 
        Qs[i] = 1.0/max(Q_i,THRESH)
        loss += y_out[ind]/max(Q_i,THRESH)
        i+=1 
    
    norm_sum = np.mean(Qs)
    if n_overlap == 0:
        print "no overlap"
        return 0
    return loss/(1.0*n*norm_sum)

def pol_disc_l2_wrapper(beta, args): 
    data = dict(args)
    t_lo = data['t_lo']; t_hi = data['t_hi']; pol = data['pol']
    tau = np.clip(np.dot(beta, data['x'].T) , t_lo, t_hi).flatten()
    data['tau'] = tau
    return data['sgn']*(off_pol_disc_evaluation(pol,**data) + LAMBDA * np.linalg.norm(beta,2))



In [ ]:
t_lo = min(T)
t_hi = max(T)
bins = np.linspace(t_lo, t_hi, n_bins)
T_binned = np.digitize(T, bins).flatten()
bin_means = [T[T_binned == i].mean() for i in range(1, n_bins)]

In [ ]:
data = { 'n': nn,'h': 5*pow((n_0*1.0/nn),.2), 'y': Y[trainind],'Q': norm_T_Q,'x_full': x[trainind,:], 'x': x[trainind,:],'x_samp': x[trainind,:], 'T_samp': T[trainind].flatten(), 'd': d, 'T': T[trainind],'t_lo': t_lo ,'t_hi': t_hi  }
data['kernel_int_func'] = epanechnikov_int
data['T_samp'] = np.array(T[trainind],order='F'); train_data['y_samp'] = Y[trainind]
data['kernel_func'] = epanechnikov_kernel
data['inds'] = trainind; data['sgn'] = 1
data['threshold'] = 0.02
data['n_bins'] = 10 
data['pol'] = discretize_tau_policy
data['tau'] = T[trainind]
off_pol_disc_evaluation(discretize_tau_policy, **data)
beta_d = np.random.uniform(size=(d,1))*4
disc_res = minimize(pol_disc_l2_wrapper, x0 = beta_d, bounds = bnds, options={'disp':True,'gtol':1e-1,'maxfun': 1000}, args=data.items())
data['tau'] = np.dot(disc_res.x, x.T)
disc_pol = discretize_tau_policy(**data)
disc_T= np.dot(disc_res.x, x.T)
np.mean( real_risk(np.dot(disc_res.x, x.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x) )

In [ ]:
import cvxpy as cvx
np.random.seed(2)

n = len(Y)
SAMP_N = 5; 
ns = [200, 400]#, 800, 1600 ]
n_0 = ns[0]
cons_evals = np.zeros([len(ns), SAMP_N]); mean_evals = np.zeros([len(ns), SAMP_N])
zero_evals = np.zeros([len(ns), SAMP_N]); oracle_evals = np.zeros([len(ns), SAMP_N]);DR_oracle_evals = np.zeros([len(ns), SAMP_N])
oracle_coefs = [ [None]*SAMP_N ] * len(ns)
DR_oracle_coefs = [ [None]*SAMP_N ] * len(ns)
bounds = [(min(T)/(0.5*d) , max(T)/np.abs(0.5*d)  ) for i in range(d) ]
bnds = tuple(tuple(x) for x in bounds)
# bounds = [(-np.mean(sampled_T)/max(np.mean(X_sim_std[:,i]), 0.25*d) , np.mean(sampled_T)/max(np.mean(X_sim_std[:,i]),d*.25) ) for i in range(d) ]
# bounds = [(-max(sim_dose)/(.25*d*np.mean(subframe[:,i])) , max(sim_dose)/(.25*d*np.mean(subframe[:,i])) ) for i in range(d) ]
print "opt coef bounds for beta: "
print bounds

n_restarts = 1
for ind, nn in enumerate(ns): 
    print nn 
    for k in range(SAMP_N): 
        trainind = np.random.choice(range(n),size = nn, replace = False)
        sub_T = np.zeros(nn)
        
        # Learn GPS from data
        lr = LinearRegression(); lr.fit(x[trainind,:], T[trainind])
        T_hat = lr.predict(x[trainind,:])
        beta_T_gps = np.dot(x[trainind,:],lr.coef_)
        resid = T[trainind] - T_hat
        # Assume normal noise
        mu_resid = np.mean(resid); sigma_resid = np.std(resid)
        def norm_T_Q_est(x,t,t_lo,t_hi): 
            return norm.pdf( t - np.dot(x, lr.coef_), loc =  mu_resid, scale = sigma_resid )
        
        data = { 'n': nn,'h': 3*pow((n_0*1.0/nn),.2), 'y': Y[trainind],'Q': norm_T_Q_est,'x_full': x[trainind,:], 'x': x[trainind,:],'x_samp': x[trainind,:], 'T_samp': T[trainind].flatten(), 'd': d, 'T': T[trainind],'t_lo': t_lo ,'t_hi': t_hi  }
        data['kernel_int_func'] = epanechnikov_int
        data['T_samp'] = np.array(T[trainind],order='F'); train_data['y_samp'] = Y[trainind]
        data['kernel_func'] = epanechnikov_kernel
        data['inds'] = trainind; data['sgn'] = 1
        data['threshold'] = 0.02
        betas = []; vals = []
        
        k_ind = 0
        while k_ind <= n_restarts: 
            k_ind += 1
            beta_d = np.random.uniform(size=d)
            LAMBDA = 0.5
            oracle_res = minimize(pol_eval_l2_wrapper, x0 = beta_d, jac = off_pol_epan_lin_l2_grad, bounds = bnds, options={'disp':True,'gtol':1e-1,'maxfun': 1000}, args=data.items())  
            betas += [oracle_res.x]
            print oracle_res.x
            vals += [oracle_res.fun]
            print oracle_res.fun
        oracle_coefs[ind][k] = betas[np.argmin(vals)] 
        oracle_evals[ind, k] = vals[np.argmin(vals)] 
        zero_T = np.zeros(nn); data['tau'] = zero_T; zero_evals[ind,k] = off_policy_evaluation(**data)
#         oracle_coefs[ind] += [ betas[np.argmin(vals)] ]
#         oracle_evals[ind, k] = vals[np.argmin(vals)]
        mu_T = np.mean(T)*np.ones(nn); data['tau'] = mu_T; mean_evals[ind,k] = off_policy_evaluation(**data)
        
        clf = RandomForestRegressor(n_estimators=5)
        clf = clf.fit(x[trainind,:],y=Y[trainind])
#         data['rf'] = clf
        y_hat = clf.predict(x[trainind,:])
        data['y'] = Y[trainind] - y_hat
        betas = []; vals = []
        k_ind = 0
        while k_ind <= n_restarts: 
            k_ind += 1
            beta_d = np.random.uniform(size=d)
            LAMBDA = 0.5
            oracle_res = minimize(pol_eval_l2_wrapper, x0 = beta_d, jac = off_pol_epan_lin_l2_grad, bounds = bnds, options={'disp':True,'gtol':1e-1,'maxfun': 1000}, args=data.items())  
            print oracle_res.x
            betas += [oracle_res.x]
            vals += [oracle_res.fun]
            print oracle_res.fun
        DR_oracle_coefs[ind][k] = betas[np.argmin(vals)] 
        DR_oracle_evals[ind, k] = vals[np.argmin(vals)] 
        
        data['n_bins'] = 10 
        off_pol_disc_evaluation(discretize_tau_policy, **data)
        
        ## Best-in-class policy via regression
        # find best in class policy 
#         d = x.shape[1]; beta = cvx.Variable(d)
#         # Implement a thresholded loss
#         cost = cvx.sum_entries(cvx.square(x[trainind,:]*beta)) # minimize y^2 
#         constraints = [beta >=min(T)/(0.5*d) ]
#         for i in range(d): 
#             constraints += [beta[i] <= max(T)/(0.5*d) ]
#         prob = cvx.Problem(cvx.Minimize(cost), constraints)
#         prob.solve('SCS')
#         print beta.value
#         prescient_beta = beta.value.flatten().T
#         prescient_beta = np.ravel(prescient_beta).T
        


$$ \beta^T X + \beta_{x,T}*X\circ T + (\beta_{x,T^2}^T x - T)^2$$

Suppose we replace T with the linear policy $\beta_{\tau}x$: 

$$ \beta^T X + \beta_{x,T}*X\circ \beta_{\tau}X + ((\beta_{x,T^2} - \beta_{\tau})^T X)^2 $$ 

If we derive wrt $\beta_{\tau x}$: 

$$ \beta_{x,T}X +2(\beta_{x,T^2}^T X - \beta_{\tau}X)X = 0 $$

$$\beta_{\tau}^T X)X =  \beta_{x,T}X +2\beta_{x,T^2}^T X $$

$$\beta_{\tau}^T  =  (\beta_{x,T}X +2\beta_{x,T^2}^T X)(X^TX)^{-1} $$

In [ ]:
def oracle_eval_wrapper(beta, args): 
    data = dict(args)
    t_lo = data['t_lo']; t_hi = data['t_hi']
    tau = np.clip(np.dot(beta, data['x'].T) , t_lo, t_hi).flatten()
    x = data['x']
    return np.mean(real_risk(np.dot(tau, x), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x))
beta_d = np.random.uniform(size=(d,1))*5
best_in_class = minimize(oracle_eval_wrapper, x0 = beta_d, bounds = bnds, options={'disp':True,'gtol':1e-1,'maxfun': 1000}, args=data.items())  
print best_in_class.x
print best_in_class.fun


In [ ]:

def real_risk(T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x):    
    n = len(T); risk = np.zeros(n)
    if np.isscalar(T):
        risk = T*beta_cons + np.dot(beta_x.T, x) + np.dot(beta_x_T.T, x*T) + (T-np.dot(beta_x_quad_T.T,x))**2
    else: 
        for i in range(len(T)): 
            risk[i] = T[i]*beta_cons + np.dot(beta_x.T, x[i,:]) + np.dot(beta_x_T.T, x[i,:]*T[i]) + (T[i]-np.dot(beta_x_quad_T.T,x[i,:]))**2#+ np.dot(beta_x_quad_T.T, (x[i,:]**2)*T[i]) + np.dot(beta_x_high_freq.T, np.sin(x[i,0:HIGH_FREQ_N]*FREQ)*T[i])
    return risk

zero_T = np.zeros(n); 
# pred_T = np.dot(oracle_coefs[1][0], x.T)
dr_T = np.dot(DR_oracle_coefs[1][0], x.T)
colors = ['b', 'g', 'r', 'y','orange']
EVALS = [zero_evals, mean_evals, DR_oracle_evals]

# use scipy minimize (1D optimize outcome wrt T )
def get_oracle_treatment(beta_cons, beta_x, beta_x_T, x): 
    res = minimize(real_risk, np.mean(T), args =(beta_cons, beta_x, beta_x_T, beta_x_quad_T, x))
    return res.x 

POLICIES = [ zero_T, mu_T, dr_T ]
lbls = ['zero treatment policy', 'mean treatment policy', 'DR treatment policy'   ]

def plot_evals(evals, POLICY, X, c, lbl, ns, SAMP_N): 
    plt.axhline(y = Y_OFFSET + np.mean(real_risk(POLICY, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x)),color=c)
    plt.scatter(ns, np.mean(evals,axis=1),label=lbl,color=c)
    error = 1.96*np.std(evals,axis=1)/np.sqrt(SAMP_N)
    plt.fill_between(ns, np.mean(evals,axis=1)-error,np.mean(evals,axis=1)+error, alpha=0.3, edgecolor=c, facecolor=c)
# plt.axhline(y = np.mean(real_risk(oracle_T beta_cons, beta_x, beta_x_T, x)))


plt.figure()
for j in range(len(ns)):
    for i in range(SAMP_N): 
        plt.hist( np.asarray(real_risk(np.dot(DR_oracle_coefs[j][i], x.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x)).flatten())

plt.figure()
[ plot_evals(EVALS[i], POLICIES[i], x, colors[i], lbls[i], ns, SAMP_N) for i in range(len(EVALS))]
for j in range(len(ns)):
    for i in range(SAMP_N): 
        plt.axhline(y = np.mean(real_risk(np.dot(oracle_coefs[j][i], x.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x)),color='r')
    for i in range(SAMP_N): 
        plt.axhline(y = np.mean(real_risk(np.dot(DR_oracle_coefs[j][i], x.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x)),color='r')
plt.axhline(y = np.mean(real_risk(np.dot(oracle_res.x, x.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x)))

plt.figure()
plt.hist(real_risk(np.dot(oracle_res.x, x.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x))
# plt.axhline(y = np.mean(real_risk(pred_T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x)),color=c)
plt.figure()
plt.hist(real_risk(zero_T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x))


# def plot_evals(evals, POLICY, X, c, lbl, ns, SAMP_N): 
#     plt.axhline(y = np.mean(real_risk(POLICY, beta_cons, beta_x, beta_x_T, x)),color=c)
#     plt.scatter(ns, np.mean(evals,axis=1),label=lbl,color=c)
#     error = 1.96*np.std(evals,axis=1)/np.sqrt(SAMP_N)
#     plt.fill_between(ns, np.mean(evals,axis=1)-error,np.mean(evals,axis=1)+error, alpha=0.5, edgecolor=c, facecolor=c)
# plt.axhline(y = np.mean(real_risk(oracle_T, beta_cons, beta_x, beta_x_T, x)))
# [ plot_evals(EVALS[i], POLICIES[i], x, colors[i], lbls[i], ns, SAMP_N) for i in range(len(EVALS))]

import pickle
plt.figure()
for i in range(len(EVALS)):  
    pickle.dump(EVALS[i], open(str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")) +lbls[i]+ 'EVALS.p', 'wb'))
    pickle.dump(POLICIES[i], open(str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")) +lbls[i]+ 'policy.p', 'wb'))
    pickle.dump(oracle_coefs, open(str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")) +lbls[i]+ 'oracle_coefs.p', 'wb'))
    pickle.dump(DR_oracle_coefs, open(str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")) +lbls[i]+ 'DR_oracle_coefs.p', 'wb'))



In [ ]:
plt.hist(T)

In [ ]:
# Impute POEM 
import sys
sys.path.append('./POEM-norm/')
import DatasetReader, Skylines

def norm_T_Q(x,t,t_lo,t_hi): 
    return norm.pdf( t - np.dot(x, TRUE_PROP_BETA), loc = 0, scale = T_SIG )

t_lo = min(T)
t_hi = max(T)
bins = np.linspace(t_lo, t_hi, n_bins)
bin_width = (t_hi-t_lo)*1.0/n_bins
T_binned = np.digitize(T, bins,right=True).flatten()
bin_means = [T[T_binned == i].mean() for i in range(1, n_bins)]
Xtrain = x[trainind,:]
ttrain = T_binned[trainind]

gps = np.asarray([bin_width*norm_T_Q(x[i,:], bin_means[ttrain[i]-1], 0.5, 0.5 ) for i in range(len(trainind))])
# all 0-1
# gps = integrated_propensity_scores
ytrain = Y[trainind]
Xtest = x

mydata = DatasetReader.BanditDataset(None,False)
mydata.trainFeatures        = np.hstack((Xtrain.copy(),np.ones((len(Xtrain),1))))
mydata.sampledLabels        = np.zeros((len(ttrain),max(ttrain)+1))
mydata.sampledLabels[range(len(ttrain)),ttrain] = 1.
mydata.trainLabels          = np.empty(mydata.sampledLabels.shape)
mydata.sampledLoss          = ytrain.copy()
mydata.sampledLoss         -= mydata.sampledLoss.min()
mydata.sampledLoss         /= mydata.sampledLoss.max()
# computed on training set 
mydata.sampledLogPropensity = np.log(gps)
#ones_like vs ones_line? 
mydata.testFeatures              = np.hstack((np.ones_like(Xtest),np.ones((len(Xtest),1))))
mydata.testLabels                = np.array([])
mydata.createTrainValidateSplit()
pool = None 
coef = None

maj = Skylines.PRMWrapper(mydata, n_iter = 1000, tol = 1e-6, minC = 0, maxC = -1, minV = 0, maxV = -1,
                            minClip = 0, maxClip = 0, estimator_type = 'Self-Normalized', verbose = True,
                            parallel = pool, smartStart = coef)
maj.calibrateHyperParams()
maj.validate()
Xtest1 = np.hstack((Xtest,np.ones((len(Xtest),1))))
rec = Xtest1.dot(maj.labeler.coef_).argmax(1)



In [ ]:
POEM_pol =  [bin_means[rec[i]-1] for i in range(n)]  
# print POEM_pol
POEM_risk = real_risk(POEM_pol, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x)
print np.mean(POEM_risk)
print np.median(POEM_risk)
plt.hist(POEM_risk)


In [ ]:
X_w_dose

DM with random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

X_w_dose = np.column_stack((x, T))
X_w_dose.shape

clf = RandomForestRegressor(n_estimators=10)

clf = clf.fit(X_w_dose[trainind,:],y=Y[trainind])
data['x_full'] = X_w_dose[trainind,:]
data['rf'] = clf
print clf.score(X_w_dose,Y)

def dm_wrapper(beta, args): 
    reg_lambda = 0.5
    data = dict(args)
    t_lo = data['t_lo']; t_hi = data['t_hi']
    x = data['x_full']
    clf = data['rf']
    tau = np.clip(np.dot( beta, x[:,:-1].T ), t_lo, t_hi).flatten()
    counterfactual_X = np.column_stack((x[:,:-1], tau))
    return (np.mean( clf.predict( counterfactual_X )) + reg_lambda * np.linalg.norm( beta, 2 )) 

dm_res = minimize(dm_wrapper, x0 = beta_d, bounds = bnds, options={'disp':True,'gtol':1e-1,'maxfun': 1000,'eps':np.ones(d)*1e-1}, args=data.items()) 
print dm_res.x



In [ ]:
np.mean(T)

In [ ]:

dm_T = np.dot(dm_res.x, x_test.T)

plt.hist(real_risk(dm_T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test))
np.mean(real_risk(dm_T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test))

In [ ]:
def get_oracle_treatment(beta_cons, beta_x, beta_x_T, beta_x_quad_T, x): 
    res = minimize(real_risk_scalar, np.mean(T), args =(beta_cons, beta_x, beta_x_T, beta_x_quad_T,x))
    return res.x 
n_test = x_test.shape[0]
oracle_T = [ get_oracle_treatment(beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test[i,:]) for i in range(n_test)]



In [ ]:
plt.hist(np.dot(disc_res.x, x_test.T))

In [ ]:
x_test.shape

In [ ]:
data['x']
data['tau'] = np.dot(disc_res.x, x_test.T)
disc_pol_binned = np.digitize(np.dot(disc_res.x, x_test.T), bins).flatten()
disc_pol = np.asarray( [bin_means[disc_pol_binned[i] - 1] for i in range(x_test.shape[0])] )
plt.hist(disc_pol)

In [ ]:

def cons_pol_wrapper(beta, args): 
    data = dict(args)
    t_lo = data['t_lo']; t_hi = data['t_hi']
    tau = beta*np.ones(data['n'])
    data['tau'] = tau
    return data['sgn']*(off_policy_evaluation(**data))
beta_t = np.random.uniform()*4
bnds_t = tuple([(-15,15)])
cons_res = minimize(cons_pol_wrapper, x0 = beta_t, bounds = bnds_t, options={'disp':True,'gtol':1e-1,'maxfun': 1000}, args=data.items())




In [ ]:
5

In [ ]:
%matplotlib inline
plt.style.use('seaborn-white')
plt.figure()

# Use the same simulated loss function to estimate differences
TRUE_ORACLE = real_risk(oracle_T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)
oracle_loss = real_risk(np.dot(best_in_class.x, x_test.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)
zero_pol = real_risk(np.zeros(n_test), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)
mu_pol = real_risk(np.ones(n_test)*np.mean(T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)
DM = real_risk(np.dot(dm_res.x, x_test.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)
DR_oracle = real_risk(np.dot(DR_oracle_coefs[1][3],x_test.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)
off_pol = real_risk(np.dot(oracle_coefs[1][1],x_test.T), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)
disc_pol_norms = real_risk(disc_pol, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)
best_constant = real_risk(cons_res.x*np.ones(n_test), beta_cons, beta_x, beta_x_T, beta_x_quad_T, x_test)


original = real_risk(T, beta_cons, beta_x, beta_x_T, beta_x_quad_T, x)

norms = [TRUE_ORACLE, off_pol,DR_oracle,DM, disc_pol_norms,best_constant, mu_pol, original ]

plt.figure(figsize=(4.5,3))
plt.ylabel('Thresholded absolute policy loss')
flierprops = dict(linestyle='-',color='black')

# prefer to turn off mean line
medianprops = dict(linestyle='', linewidth=0, color='firebrick')
f=plt.figure(figsize=(4.5,2.5))
bp_dict = plt.boxplot(norms,sym='', showmeans = True, meanline=True,medianprops=medianprops)
plt.ylabel('Outcome (loss)')
for whisker in bp_dict['whiskers']:
    whisker.set(color='#000000',linestyle='solid')
for box in bp_dict['boxes']:
    # change outline color
    box.set( color='#000000')
for ind,line in enumerate(bp_dict['means']):
    # get position data for means line
    xx, y = line.get_xydata()[1] # top of means line
    if ind == 7: 
        plt.text(xx-1, 30, '%.1f' % y, horizontalalignment='center',size=13) # draw above, centered
    # overlay median value
    else:
        plt.text(xx+0.2, -35, '%.1f' % y, horizontalalignment='center',size=13) # draw above, centered
# top = 60
# bottom = 0
# plt.ylim(bottom, top)
# plt.title('Boxplot of distances between policy-recommended doses and therapeutic doses',y = 1.2)
# plt.axhline(y=np.mean(mean_dose_pol_norm), xmin=0,xmax=1,color='g', ls='dashdot',linewidth=0.7)
# plt.axhline(y=0, xmin=0,xmax=1,color='g',alpha=0.3, ls='dashed',linewidth=0.7)
plt.xticks([1, 2, 3,4,5,6,7,8], ['best o.o.c.', 'CPO', 'DR CPO', 'DM','Disc. CPO','CPE, cons.', 'Mean','Original'])
plt.xticks(rotation=25)
plt.ylim((-40,40))
ax = plt.subplot(111)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
f.savefig("boxplot_risks.pdf", bbox_inches='tight')




plt.show()

f= plt.figure()
plt.violinplot(norms)
plt.xticks([1, 2, 3,4,5,6,7,8], ['Best o.o.c.','OPE', 'DR OPE','DM','Disc. OPE','OPE, cons.','Mean','Original'],size = 13)
plt.xticks(rotation=25)
plt.ylim((-90,70))
for ind,line in enumerate(bp_dict['means']):
    # get position data for means line
    xx, y = line.get_xydata()[1] # top of means line
    if ind == 6: 
        plt.text(xx+0.05, y+3.5, '%.1f' % y, horizontalalignment='center',size=13) # draw above, centered
    # overlay median value
    else:
        plt.text(xx+0.24, y+3.5, '%.1f' % y, horizontalalignment='center',size=13) # draw above, centered

        
f.savefig("violin_plot_risks.pdf", bbox_inches='tight')


In [ ]:
5

In [ ]:
from statsmodels.nonparametric.kernel_regression import KernelReg
from statsmodels.nonparametric.kernel_density import EstimatorSettings


settings = EstimatorSettings(efficient = True, randomize=True)
list_data = [ x[trainind,i] for i in range(d) ] + [T[trainind]]
kernel_regr = KernelReg(endog = [Y[trainind].reshape([len(trainind),1])], exog=list_data, var_type=['c']*len(list_data),bw='aic',defaults = settings )

In [ ]:
kernel_regr